In [1]:
import os
import torch
import argparse
import numpy as np
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch_geometric.nn import GraphUNet
from utils import *
from model import *

seed=816
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [2]:
# ---------------------
# parameters
# ---------------------
lr = 2e-3
epochs = 100
batch_size = 1
pos_weights = 5
path = '../../../data/2023-graph-conflation/'
path = 'D:graph-conflation-data/'

# ---------------------
# load data
# ---------------------
print('Load Datasets...')
files = os.listdir(path+'/graphs/osm/')
train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

# make datasets
train_data = GraphDataset(path, train_files)
val_data = GraphDataset(path, val_files)
test_data = GraphDataset(path, test_files)

# data loader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Load Datasets...


In [14]:
# ---------------------
#  models
# ---------------------
print('Load Model...')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
model_osm = GraphUNet(2,64,128,3).to(device)
model_sdot = GraphUNet(2,64,128,3).to(device)
optimizer = torch.optim.Adam(list(model_osm.parameters()) + list(model_sdot.parameters()), lr=lr)
criterion= nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weights))
es = EarlyStopping(tolerance=10)

Load Model...


In [15]:
for i, batch in enumerate(tqdm(train_dataloader)):
    
    # load data
    graph_osw, graph_osm, graph_sdot, osw_x, osm_x, sdot_x = batch
    graph_osw = graph_osw.squeeze_(0).to(device)
    labels = graph_osw.flatten()
    graph_osm = graph_osm.squeeze_(0).to(device)
    graph_sdot = graph_sdot.squeeze_(0).to(device)
    osw_x = osw_x.squeeze_(0).to(device)
    osm_x = osm_x.squeeze_(0).to(device)
    sdot_x = sdot_x.squeeze_(0).to(device)
    break

  0%|                                                                                         | 0/5670 [00:00<?, ?it/s]


In [16]:
import torch

In [8]:
a = osm_x.to_sparse()
b = graph_osm.to_sparse()

In [3]:
# ----------------
# Training
# ----------------
model_osm.train()
model_sdot.train()
Train(
    train_dataloader, 
    model_osm, 
    model_sdot, 
    optimizer, 
    criterion,
    device
)

  4%|██▊                                                                            | 201/5670 [00:04<02:03, 44.41it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\binha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\binha\AppData\Local\Temp\ipykernel_12356\2598389665.py", line 6, in <module>
    Train(
  File "C:\Users\binha\Documents\research\bill-research\2023-urban-graph-conflation\model\utils.py", line 91, in Train
    out_osm = model_osm(osm_x, graph_osm)
  File "C:\Users\binha\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "C:\Users\binha\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "C:\Users\binha\Documents\research\bill-research\2023-urban-graph-conflation\model\model.py", line 26, in forward
    x = self.conv2(x, edge_index)
  File "C:\Users\binha\anaconda3\lib\site-packages\torch\nn\modules\module.py"

TypeError: object of type 'NoneType' has no len()